<p align="center">
  <img src="https://user-images.githubusercontent.com/90031508/183531098-494a5819-7714-4f72-8ff8-d038982eb5f0.png" alt="Water Oracle logo"/>
</p>



This Work is adapted from 'Tensorflow example workflows', 
https://developers.google.com/earth-engine/guides/tf_examples examples.
Copyright 2020 Google LLC. https://www.apache.org/licenses/LICENSE-2.0.

Please run this notebook on google colab (pro+)

<table class="ee-notebook-buttons" align="left"><td>
<a target="_blank"  href="https://colab.research.google.com/drive/1FqVci8loCj-C0Zi-SynCM4IOqu1b_ihJ?usp=sharing">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a>
</td><td>
<a target="_blank"  href="https://github.com/ese-msc-2021/irp-kl121"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a></td></table>

# Introduction


## Prerequisites
- Google account and logins
- Google colab subscription with pro or pro+ is optional but would help with long runtime
- Google cloud platform account in order to use google cloud bucket. (Note that you would need sufficient funds to store large amount of models and training data.)
- Wandb.ai account which is free of charge

## What is this notebook?

The two main purpose of this notebook is
- <b> Exporting training data globally to google cloud bucket</b>: use earth engine's package to export multiple training patches that will be used to train models in the other notebook (TrainModels.ipynb)
- <b>Predicting and exporting images to earth engine asset:</b> Load the saved model generated from TrainModels.ipynb and predict the corresponding images.

## Creating Packages

Creating the tools packages that will be used throughout the notebook. The package includes 
- metrics_.py
- config.py
- preprocessing.py
- sampling.py

In [ ]:
PACKAGE_PATH = 'tools'

!ls -l
!mkdir {PACKAGE_PATH}
!touch {PACKAGE_PATH}/__init__.py
!ls -l {PACKAGE_PATH}

total 4
drwxr-xr-x 1 root root 4096 Aug  3 20:21 sample_data
total 0
-rw-r--r-- 1 root root 0 Aug  8 13:10 __init__.py


In [ ]:
%%writefile {PACKAGE_PATH}/metrics_.py

from keras import backend as K
import tqdm.notebook as tq
import numpy as np
import tensorflow as tf
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score

CONFIG = None

__all__ = ["f1", "custom_accuracy", "MetricCalculator", "MetricCalculator_multiview_2", "MetricCalculator_multiview_3", "MetricCalculator_NDWI", "ndwi_threashold"]

def f1(y_true, y_pred):
    """
    The function is used as tensorflow metrics when training. It takes in the ground truth and the
    model predicted result and evaluate the F1 score. This is an experimental function and should not be used as
    further model training metric.

    Parameters
    ----------
    y_true : tf.tensor
    y_pred : tf.tensor

    Returns
    ----------
    F1 score in keras backend

    Notes
    -----
    This function is flawed because keras calculates the metrics batchwise 
    which is why F1 metric is removed from keras. To properly calulate the F1 score, we can use the callback function
    or manually calculate F1 score after the model has finished training. The latter is chosen and this could be seen
    in MetricCalculator, MetricCalculator_multiview_2 and MetricCalculator_multiview_3.
  
    The reason this function is kept is because the model was initially trained with these metrics and
    stored in the google cloud bucket. To retrieve the models these metrics must be passed inorder to retrieve the model.
    Since the model is optimize on the loss rather than the metrics, the incorrect metric would not effect the model
    training process. The code is obtained/modified from:

    https://stackoverflow.com/questions/43547402/how-to-calculate-f1-macro-in-keras

    https://neptune.ai/blog/implementing-the-macro-f1-score-in-keras
    """
    def recall(y_true, y_pred):
        """
        Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """
        Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


def custom_accuracy(y_true, y_pred):
    """
    The function is used as tensorflow metrics when training. It takes in the ground truth and the
    model predicted result and evaluate the accuracy score. This is an experimental function and should not be used as
    further model training metric.

    Parameters
    ----------
    y_true : tf.tensor
    y_pred : tf.tensor

    Returns
    ----------
    accuracy score in keras backend

    Notes
    -----
    This function is modified from the F1 metric above to fit the definition of accuracy. However, tensorflow's
    "categorical_accuracy" is used instead. The accuracy metric would also be recalculated again in 
    MetricCalculator, MetricCalculator_multiview_2 and MetricCalculator_multiview_3.
  
    The reason this function is kept is because the model was initially trained with these metrics and
    stored in the google cloud bucket. To retrieve the models these metrics must be passed inorder to retrieve the model.
    Since the model is optimize on the loss rather than the metrics, the incorrect metric would not effect the model
    training process. The code is obtained/modified from:

    https://stackoverflow.com/questions/43547402/how-to-calculate-f1-macro-in-keras

    https://neptune.ai/blog/implementing-the-macro-f1-score-in-keras
    """
    # total_data = K.int_shape(y_true) + K.int_shape(y_pred)
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    true_negatives = K.sum(K.round(K.clip(1 - y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    total_data = - true_positives + true_negatives + possible_positives + predicted_positives
    return (true_positives + true_negatives) / (total_data + K.epsilon())



def MetricCalculator(model, test_data, total_steps):
  """
  This function takes in the feature stack model loaded from google cloud bucket, the test_data which is the tensor object and
  the number of steps and returns the metrics including accuracy, recall, precision and F1

  Parameters
  ----------
  model : keras.engine.functional.Functional
  test_data : RepeatDataset with tf.float32
  total_steps : int/float

  Returns
  ----------
  Returns the precision, recall, f1, accuracy metric based on the model performance.

  Notes
  -----
  This function should be used instead of the F1, custom_accuracy written above. The code is obtained/modified from:

  https://stackoverflow.com/questions/43547402/how-to-calculate-f1-macro-in-keras

  https://neptune.ai/blog/implementing-the-macro-f1-score-in-keras
  """
  pred = []
  true = []
  pbar = tq.tqdm(total=total_steps)
  for steps, data in enumerate(test_data):
    # print(f'Number of steps: {steps}', end = "\r")
    pbar.update(1)
    if steps == total_steps:
      break
    input = data[0]
    y_true = data[1]
    y_pred = np.rint(model.predict(input))
    y_true = np.reshape(y_true, (256*256,2))
    y_pred = np.reshape(y_pred, (256*256,2))
    pred.append(y_pred)
    true.append(y_true)


  f1_macro = f1_score(np.reshape(true, (total_steps*65536, 2)), np.reshape(pred, (total_steps*65536, 2)), average="macro")
  recall_macro= recall_score(np.reshape(true, (total_steps*65536, 2)), np.reshape(pred, (total_steps*65536, 2)), average="macro")
  precision_macro = precision_score(np.reshape(true, (total_steps*65536, 2)), np.reshape(pred, (total_steps*65536, 2)), average="macro")
  accuracy = accuracy_score(np.reshape(true, (total_steps*65536, 2)), np.reshape(pred, (total_steps*65536, 2)))

  print("precision_macro: ", precision_macro)
  print("recall_macro: ", recall_macro)
  print("F1_macro_Score: : ", f1_macro)
  print("Accuracy: ", accuracy)

  return precision_macro, recall_macro, f1_macro, accuracy



def MetricCalculator_multiview_2(model, test_data, total_steps):
  """
  This function takes in the multiview-2 model loaded from google cloud bucket, the test_data which is the tensor object and
  the number of steps and returns the metrics including accuracy, recall, precision and F1

  Parameters
  ----------
  model : keras.engine.functional.Functional
  test_data : RepeatDataset with tf.float32
  total_steps : int/float

  Returns
  ----------
  Returns the precision, recall, f1, accuracy metric based on the model performance.

  Notes
  -----
  This function should be used instead of the F1, custom_accuracy written above. The code is obtained/modified from:

  https://stackoverflow.com/questions/43547402/how-to-calculate-f1-macro-in-keras

  https://neptune.ai/blog/implementing-the-macro-f1-score-in-keras
  """
  pbar = tq.tqdm(total=total_steps)
  pred = []
  true = []
  for steps, data in enumerate(test_data):
    pbar.update(1)
    if steps >= total_steps:
      break
    input = data[0]
    x1, x2 = tf.split(input, [len(CONFIG.BANDS1),len(CONFIG.BANDS2)], 3)
    y_true = data[1]
    y_pred = np.rint(model.predict([x1, x2]))
    y_true = np.reshape(y_true, (256*256,2))
    y_pred = np.reshape(y_pred, (256*256,2))
    pred.append(y_pred)
    true.append(y_true)
  f1_macro = f1_score(np.reshape(true, (total_steps*65536, 2)), np.reshape(pred, (total_steps*65536, 2)), average="macro")
  recall_macro= recall_score(np.reshape(true, (total_steps*65536, 2)), np.reshape(pred, (total_steps*65536, 2)), average="macro")
  precision_macro = precision_score(np.reshape(true, (total_steps*65536, 2)), np.reshape(pred, (total_steps*65536, 2)), average="macro")
  accuracy = accuracy_score(np.reshape(true, (total_steps*65536, 2)), np.reshape(pred, (total_steps*65536, 2)))

  print("precision_macro: ", precision_macro)
  print("recall_macro: ", recall_macro)
  print("F1_macro_Score: : ", f1_macro)
  print("Accuracy: ", accuracy)

  return precision_macro, recall_macro, f1_macro, accuracy

def MetricCalculator_multiview_3(model, test_data, total_steps):
  """
  This function takes in the multiview-3 model loaded from google cloud bucket, the test_data which is the tensor object and
  the number of steps and returns the metrics including accuracy, recall, precision and F1

  Parameters
  ----------
  model : keras.engine.functional.Functional
  test_data : RepeatDataset with tf.float32
  total_steps : int/float

  Returns
  ----------
  Returns the precision, recall, f1, accuracy metric based on the model performance.

  Notes
  -----
  This function should be used instead of the F1, custom_accuracy written above. The code is obtained/modified from:

  https://stackoverflow.com/questions/43547402/how-to-calculate-f1-macro-in-keras

  https://neptune.ai/blog/implementing-the-macro-f1-score-in-keras
  """
  pbar = tq.tqdm(total=total_steps)
  pred = []
  true = []
  for steps, data in enumerate(test_data):
    pbar.update(1)
    if steps >= total_steps:
      break
    input = data[0]
    x1, x2, x3 = tf.split(input, [len(CONFIG.BANDS1),len(CONFIG.BANDS2),len(CONFIG.BANDS3)], 3)
    y_true = data[1]
    y_pred = np.rint(model.predict([x1, x2, x3]))
    y_true = np.reshape(y_true, (256*256,2))
    y_pred = np.reshape(y_pred, (256*256,2))
    pred.append(y_pred)
    true.append(y_true)
  f1_macro = f1_score(np.reshape(true, (total_steps*65536, 2)), np.reshape(pred, (total_steps*65536, 2)), average="macro")
  recall_macro= recall_score(np.reshape(true, (total_steps*65536, 2)), np.reshape(pred, (total_steps*65536, 2)), average="macro")
  precision_macro = precision_score(np.reshape(true, (total_steps*65536, 2)), np.reshape(pred, (total_steps*65536, 2)), average="macro")
  accuracy = accuracy_score(np.reshape(true, (total_steps*65536, 2)), np.reshape(pred, (total_steps*65536, 2)))

  print("precision_macro: ", precision_macro)
  print("recall_macro: ", recall_macro)
  print("F1_macro_Score: : ", f1_macro)
  print("Accuracy: ", accuracy)

  return precision_macro, recall_macro, f1_macro, accuracy


def ndwi_threashold(B3, B5):
  """
  This function takes in bands 3 and bands 5 from the landsat imagery and returns the tuple prediction of
  whether there is water present or not. The threashold is set at 0.

  Parameters
  ----------
  test_data : RepeatDataset with tf.float32
  total_steps : int/float

  Returns
  ----------
  tuple of whether there is water or not
  """
  ndwi = (B3-B5)/(B3+B5)
  if ndwi > 0:
    return 0, 1
  else:
    return 1, 0

def MetricCalculator_NDWI(test_data, total_steps):
  """
  This function takes in the test_data which is the tensor object and
  the number of steps and returns the metrics including accuracy, recall, precision and F1
  for NDWI performance.

  Parameters
  ----------
  test_data : RepeatDataset with tf.float32
  total_steps : int/float

  Returns
  ----------
  Returns the precision, recall, f1, accuracy metric based on the NDWI performance
  """
  pred = []
  true = []
  pbar = tq.tqdm(total=total_steps)
  for steps, data in enumerate(test_data):
    # print(f'Number of steps: {steps}', end = "\r")
    pbar.update(1)
    if steps == total_steps:
      break
    input = data[0]
    y_true = data[1]
    input = np.reshape(input, (256*256,2))
    y_pred = []
    for i in range(256*256):
      B3, B5 = input[i]
      first, second = ndwi_threashold(B3, B5)
      y_pred.append([first, second])
    y_true = np.reshape(y_true, (256*256,2))
    y_pred = np.reshape(y_pred, (256*256,2))
    pred.append(y_pred)
    true.append(y_true)


  f1_macro = f1_score(np.reshape(true, (total_steps*65536, 2)), np.reshape(pred, (total_steps*65536, 2)), average="macro")
  recall_macro= recall_score(np.reshape(true, (total_steps*65536, 2)), np.reshape(pred, (total_steps*65536, 2)), average="macro")
  precision_macro = precision_score(np.reshape(true, (total_steps*65536, 2)), np.reshape(pred, (total_steps*65536, 2)), average="macro")
  accuracy = accuracy_score(np.reshape(true, (total_steps*65536, 2)), np.reshape(pred, (total_steps*65536, 2)))

  print("precision_macro: ", precision_macro)
  print("recall_macro: ", recall_macro)
  print("F1_macro_Score: : ", f1_macro)
  print("Accuracy: ", accuracy)

  return precision_macro, recall_macro, f1_macro, accuracy

In [ ]:
%%writefile {PACKAGE_PATH}/config.py

import tensorflow as tf
from . import metrics_

__all__ = ["configuration"]

class configuration:
  """
  In each experiment, the combinations of satellite's bands that is used to train the neural network is different.
  Also the way to train the neural network is also different, whether it is feature stack, multiview learning with two
  or three perceptrons. As each experiment has different settings, it is important to store them and reuse this
  throughout the project. This class enables user to store the settings and reuse the settings.
  """
  def __init__(self, PROJECT_TITLE, BANDS1, TRAIN_SIZE, EVAL_SIZE, BANDS2=[], BANDS3=[], country="TH", image=None, sam_arr=None, type_=1, LOSS="categorical_crossentropy", EPOCHS=10, BATCH_SIZE = 16, dropout_prob=0.3):
    """

    Initialising/storing the parameters to use later

    Parameters
    ----------
    PROJECT_TITLE : string
    BANDS1 : list
    TRAIN_SIZE : int/float
    EVAL_SIZE : int/float
    BANDS2 : list
    BANDS3 : list
    country : string
    image : ee.image.Image
    sam_arr : ee.image.Image
    type : int/float

    """
    if type_ == 1:
      self.type_ = "fs"
    elif type_ == 2:
      self.type_ = "m2"
    elif type_ == 3:
      self.type_ = "m3"
    else:
      self.type_ = None
    self.country = country
    self.PROJECT_TITLE = PROJECT_TITLE
    self.BANDS1 = BANDS1
    self.BANDS2 = BANDS2
    self.BANDS3 = BANDS3
    self.BUCKET = "geebucketwater"
    self.FOLDER = f'{self.type_}_{self.country}_Cnn_{self.PROJECT_TITLE}'
    self.TRAIN_SIZE = TRAIN_SIZE
    self.EVAL_SIZE = EVAL_SIZE
    self.BUCKET = "geebucketwater"
    self.TRAINING_BASE = f'training_patches'
    self.EVAL_BASE = f'eval_patches'
    self.TEST_BASE = f'test_patches'
    self.RESPONSE = 'water'
    self.BANDS = BANDS1 + BANDS2 + BANDS3 
    self.FEATURES = BANDS1 + BANDS2 + BANDS3 + [self.RESPONSE]
    # Specify the size and shape of patches expected by the model.
    self.KERNEL_SIZE = 256
    self.KERNEL_SHAPE = [self.KERNEL_SIZE, self.KERNEL_SIZE]
    self.COLUMNS = [
      tf.io.FixedLenFeature(shape=self.KERNEL_SHAPE, dtype=tf.float32) for k in self.FEATURES
    ]
    self.FEATURES_DICT = dict(zip(self.FEATURES, self.COLUMNS))
    # Specify model training parameters.
    self.BATCH_SIZE = BATCH_SIZE
    self.EPOCHS = EPOCHS
    self.BUFFER_SIZE = 2000
    self.OPTIMIZER = 'adam'
    self.LOSS = LOSS
    self.dropout_prob = dropout_prob
    self.METRICS = ['AUC', "categorical_accuracy", metrics_.f1]
    self.image = image
    self.sam_arr = sam_arr



In [ ]:
%%writefile {PACKAGE_PATH}/preprocessing.py

import tensorflow as tf
import ee

__all__ = ["Preprocessor", "maskL8sr", "EnsureTwodigit", "GenSeasonalDatesMonthly"]

class Preprocessor:
  """
  Class that preprocessese and returns the training, evaluation and testing data from google cloud bucket
  """
  def __init__(self, config):
    self.config = config

  def parse_tfrecord(self, example_proto):
    """
    The parsing function Read a serialized example into the structure defined by FEATURES_DICT.
  
    Parameters
    ----------
    example_proto: a serialized Example

    Returns
    ----------
    A dictionary of tensors, keyed by feature name.

    Notes
    -----
    The code is obtained/modified from:

    https://github.com/google/earthengine-api/blob/master/python/examples/ipynb/UNET_regression_demo.ipynb
    """
    return tf.io.parse_single_example(example_proto, self.config.FEATURES_DICT)


  def to_tuple(self, inputs):
    """
    Function to convert a dictionary of tensors to a tuple of (inputs, outputs).
    Turn the tensors returned by parse_tfrecord into a stack in HWC shape.
    Parameters
    ----------
    inputs: A dictionary of tensors, keyed by feature name.

    Returns
    ----------
    A tuple of (inputs, outputs).

    Notes
    -----
    The code is obtained/modified from:

    https://github.com/google/earthengine-api/blob/master/python/examples/ipynb/UNET_regression_demo.ipynb
    """
    inputsList = [inputs.get(key) for key in self.config.FEATURES]
    stacked = tf.stack(inputsList, axis=0)
    # Convert from CHW to HWC
    stacked = tf.transpose(stacked, [1, 2, 0])
    return stacked[:,:,:len(self.config.BANDS)], tf.reshape(tf.one_hot(tf.cast(stacked[:,:,len(self.config.BANDS):], tf.int32), depth=2),[256,256,2])


  def get_dataset(self, pattern):
    """
    Function to read, parse and format to tuple a set of input tfrecord files.
    Get all the files matching the pattern, parse and convert to tuple.
    Parameters
    ----------
    pattern: A file pattern to match in a Cloud Storage bucket.

    Returns
    ----------
    A tf.data.Dataset

    Notes
    -----
    The code is obtained/modified from:

    https://github.com/google/earthengine-api/blob/master/python/examples/ipynb/UNET_regression_demo.ipynb
    """
    try:
      glob = tf.io.gfile.glob(pattern)
    except:
      # print("the bucket you specified doesn't exist")
      return "the bucket you specified doesn't exist"
    # glob = tf.io.gfile.glob(pattern)
    if glob == []:
      return "the path you specified doesn't have the data"
    dataset = tf.data.TFRecordDataset(glob, compression_type='GZIP')
    dataset = dataset.map(self.parse_tfrecord, num_parallel_calls=5)
    dataset = dataset.map(self.to_tuple, num_parallel_calls=5)
    return dataset

  def get_training_dataset(self, location):
    """
    Get the preprocessed training dataset
    Parameters
    ----------
    location: string

    Returns
    ----------
    A tf.data.Dataset of training data.

    Notes
    -----
    The code is obtained/modified from:

    https://github.com/google/earthengine-api/blob/master/python/examples/ipynb/UNET_regression_demo.ipynb
    """
    glob = 'gs://' + self.config.BUCKET + '/' + location + "training_patches_" + '*'
    # print(glob)
    dataset = self.get_dataset(glob)
    dataset = dataset.shuffle(self.config.BUFFER_SIZE).batch(self.config.BATCH_SIZE).repeat()
    return dataset

  def get_training_dataset_for_testing(self, location):
    """
    Get the preprocessed training dataset for testing
    Parameters
    ----------
    location: string

    Returns
    ----------
    A tf.data.Dataset of training data.

    Notes
    -----
    The code is obtained/modified from:

    https://github.com/google/earthengine-api/blob/master/python/examples/ipynb/UNET_regression_demo.ipynb
    """
    glob = 'gs://' + self.config.BUCKET + '/' + location + "training_patches_" + '*'
    # print(glob)
    dataset = self.get_dataset(glob)
    if type(dataset) == str:
      return dataset
    dataset = dataset.batch(1).repeat()
    return dataset

  def get_eval_dataset(self, location):
    """
    Get the preprocessed evaluation dataset
    Parameters
    ----------
    location: string

    Returns
    ----------
    A tf.data.Dataset of evaluation data.

    Notes
    -----
    The code is obtained/modified from:

    https://github.com/google/earthengine-api/blob/master/python/examples/ipynb/UNET_regression_demo.ipynb
    """
    glob = 'gs://' + self.config.BUCKET + '/' + location + "eval_patches_" + '*'
    # print(glob)
    dataset = self.get_dataset(glob)
    if type(dataset) == str:
      return dataset
    dataset = dataset.batch(1).repeat()
    return dataset

  # print(iter(evaluation.take(1)).next())

  def get_test_dataset(self, location, test_base):
    """
    Get the preprocessed testing dataset
    Parameters
    ----------
    location: string

    Returns
    ----------
    A tf.data.Dataset of testing data.

    Notes
    -----
    The code is obtained/modified from:

    https://github.com/google/earthengine-api/blob/master/python/examples/ipynb/UNET_regression_demo.ipynb
    """
    glob = 'gs://' + self.config.BUCKET + '/' + location + test_base + '*'
    # print(glob)
    dataset = self.get_dataset(glob)
    if type(dataset) == str:
      return dataset
    dataset = dataset.batch(1).repeat()
    return dataset

def maskL8sr(image):
    """
    Get the landsat-8 image and returned a cloud masked image
    ----------
    image: ee.image.Image

    Returns
    ----------
    A maksed landsat-8 ee.image.Image

    Notes
    -----
    The code is obtained/modified from:

    https://github.com/google/earthengine-api/blob/master/python/examples/ipynb/UNET_regression_demo.ipynb
    """
    BANDS = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7']
    cloudShadowBitMask = ee.Number(2).pow(3).int()
    cloudsBitMask = ee.Number(2).pow(5).int()
    qa = image.select('pixel_qa')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(
      qa.bitwiseAnd(cloudsBitMask).eq(0))
    return image.updateMask(mask).select(BANDS).divide(10000)


def EnsureTwodigit(number):
  """
  Transform the input month into string in the
  correct format for date and time.
  ----------
  number: int

  Returns
  ----------
  months in string.

  """
  if number > 12:
    return str(12)
  if number < 10:
    return "0"+str(number)
  else:
    return str(number)

def GenSeasonalDatesMonthly(start, end, month_frequency = 3):
  """
  Given two dictionary containing the key month and year,
  return two arrays that contains the time between the 
  interval of start and end.
  ----------
  start: dict
  end: dict

  Returns
  ----------
  Two arrays containing the time elapsed between start and end

  """
  diff_year = end["year"] - start["year"]
  diff_month = end["month"] - start["month"]
  starts = []
  ends = []
  first_data = str(start["year"]) + "-" + EnsureTwodigit(start["month"]) + "-01"
  if diff_year > 0:
    return "please insert the same year"
  else:
    for i in range(round(diff_month/month_frequency)):
      first_data = str(start["year"]) + "-" + EnsureTwodigit(start["month"] + month_frequency * i) + "-01"
      second_data = str(start["year"]) + "-" + EnsureTwodigit(start["month"] + month_frequency * i + month_frequency) + "-01"
      starts.append(first_data)
      ends.append(second_data)
  return starts, ends


In [ ]:
%%writefile {PACKAGE_PATH}/sampling.py

import ee

__all__ = ["Training_task", "Eval_task", "Testing_task"]

def Training_task(trainingPolys, n, N, arrays, setting, foldername):
  """
  Exporting Training data to google cloud bucket
  Parameters
  ----------
  trainingPolys : ee.featurecollection.FeatureCollection
  n : int/float
  N : int/float
  arrays: ee.image.Image
  setting: tools.config.configuration
  foldername : string

  Returns
  ----------
  A tf.data.Dataset of testing data.

  Notes
  -----
  The code is obtained/modified from:

  https://github.com/google/earthengine-api/blob/master/python/examples/ipynb/UNET_regression_demo.ipynb
  """
  trainingPolysList = trainingPolys.toList(trainingPolys.size())
  # Export all the training data (in many pieces), ith one task 
  # per geometry.
  for g in range(trainingPolys.size().getInfo()):
    geomSample = ee.FeatureCollection([])
    for i in range(n):
      sample = arrays.sample(
        region = ee.Feature(trainingPolysList.get(g)).geometry(), 
        scale = 30,
        numPixels = N / n, # Size of the shard.
        seed = i,
        tileScale = 8
      )
      geomSample = geomSample.merge(sample)
    
    desc = setting.TRAINING_BASE + '_g' + str(g)
    task = ee.batch.Export.table.toCloudStorage(
      collection = geomSample,
      description = desc,
      bucket = setting.BUCKET,
      fileNamePrefix = foldername + '/' + desc,
      fileFormat = 'TFRecord',
      selectors = setting.BANDS + [setting.RESPONSE]
    )
    task.start()


def Eval_task(evalPolys, n, N, arrays, setting, foldername):
  """
  Exporting Evaluating data to google cloud bucket
  Parameters
  ----------
  evalPolys : ee.featurecollection.FeatureCollection
  n : int/float
  N : int/float
  arrays: ee.image.Image
  setting: tools.config.configuration
  foldername : string

  Returns
  ----------
  A tf.data.Dataset of testing data.

  Notes
  -----
  The code is obtained/modified from:

  https://github.com/google/earthengine-api/blob/master/python/examples/ipynb/UNET_regression_demo.ipynb
  """
  evalPolysList = evalPolys.toList(evalPolys.size())
  # Export all the evaluation data.
  for g in range(evalPolys.size().getInfo()):
    geomSample = ee.FeatureCollection([])
    for i in range(n):
      sample = arrays.sample(
        region = ee.Feature(evalPolysList.get(g)).geometry(), 
        scale = 30,
        numPixels = N / n,
        seed = i,
        tileScale = 8
      )
      geomSample = geomSample.merge(sample)

    desc = setting.EVAL_BASE + '_g' + str(g)
    task = ee.batch.Export.table.toCloudStorage(
      collection = geomSample,
      description = desc,
      bucket = setting.BUCKET,
      fileNamePrefix = foldername + '/' + desc,
      fileFormat = 'TFRecord',
      selectors = setting.BANDS + [setting.RESPONSE]
    )
    task.start()


def Testing_task(testPolys, n, N, arrays, setting, foldername, Test_base):
  """
  Exporting Testing data to google cloud bucket
  Parameters
  ----------
  testPolys : ee.featurecollection.FeatureCollection
  n : int/float
  N : int/float
  arrays: ee.image.Image
  setting: tools.config.configuration
  foldername : string

  Returns
  ----------
  A tf.data.Dataset of testing data.

  Notes
  -----
  The code is obtained/modified from:

  https://github.com/google/earthengine-api/blob/master/python/examples/ipynb/UNET_regression_demo.ipynb
  """
  # Export all the test data.
  testPolysList = testPolys.toList(testPolys.size())
  for g in range(testPolys.size().getInfo()):
    geomSample = ee.FeatureCollection([])
    for i in range(n):
      sample = arrays.sample(
        region = ee.Feature(testPolysList.get(g)).geometry(), 
        scale = 30,
        numPixels = N / n,
        seed = i,
        tileScale = 8
      )
      geomSample = geomSample.merge(sample)

    desc = Test_base + '_g' + str(g)
    task = ee.batch.Export.table.toCloudStorage(
      collection = geomSample,
      description = desc,
      bucket = setting.BUCKET,
      fileNamePrefix = foldername + '/' + desc,
      fileFormat = 'TFRecord',
      selectors = setting.BANDS + [setting.RESPONSE]
    )
    task.start()



## Authentication

Authentication with google colab, earth engine api and google cloud bucket is required before proceeding.

In [ ]:
# Cloud authentication.
from google.colab import auth
auth.authenticate_user()

# Import, authenticate and initialize the Earth Engine library.
import ee
ee.Authenticate()
ee.Initialize()

project_id = 'coastal-cell-299117'
!gcloud config set project {project_id}

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=rLDSGqotVLeX_d2E-hBAYK73KxL67meYNBy6YluSQck&tc=K8xeBpSrZfPT-S-32jiY-kM8sMFGElOx6LhSTf0fPWw&cc=1y_ear7IfAKqFnsVpmA6V6-VNSBMM5vIyyvUema-rV0

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AdQt8qgYWT53N_MAQ2UF2cbkUW6b9lTAHNSSUCeICkSX0cD8IcfmiSBluRo

Successfully saved authorization token.
Updated property [core/project].


Import other required library

In [ ]:
from importlib import reload
import tensorflow as tf
import folium
from pprint import pprint
# reload(preprocessing) # Uncomment this line to rerun the modified packages
from tools import preprocessing, config, metrics_, sampling

# Exporting data to Google Cloud bucket

This section is dedicated entirely to exporting training, evaluating and testing data to google cloud bucket.

Initially, the required data is imported from the earth engine repository.

- The data used includes:
Sentinel-1 Data (10m) 
https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S1_GRD?hl=en

- USG’s Landsat-8 Collection 1 and Tier 1 (30m) and the cloud is masked
https://developers.google.com/earth-engine/guides/landsat
- NASADEM: NASA NASADEM Digital Elevation (30m) https://developers.google.com/earth-engine/datasets/catalog/NASA_NASADEM_HGT_001
- JRC-Monthly Water history (30m) and the region which includes "no data" is masked out https://developers.google.com/earth-engine/datasets/catalog/JRC_GSW1_3_MonthlyHistory

Date used to train: 1/1/2018 to 1/2/2018

In [ ]:
start_date =  "2018-07-01"
end_date = "2018-08-01"
# Sentinel-1 Data (10m)
S1 = ee.ImageCollection('COPERNICUS/S1_GRD') \
        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
        .filterDate(start_date, end_date) \

S1A = S1.median()
S1 = S1.select('VV', 'VH').median()

# USG’s Landsat-8 Collection 1 and Tier 1 (30m)
l8sr = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').filterDate(start_date, end_date)

# Cloud masking function.
L8SR = l8sr.map(preprocessing.maskL8sr).median()

# NASADEM: NASA NASADEM Digital Elevation (30m)
elevation = ee.Image('NASA/NASADEM_HGT/001').select('elevation');
slope = ee.Terrain.slope(elevation);
aspect = ee.Terrain.aspect(elevation);

# JRC-Monthly Water history (30m)
waterdata = ee.ImageCollection('JRC/GSW1_3/MonthlyHistory').filterDate(start_date, end_date).median()
watermask = waterdata.select("water")
mask = watermask.gt(0) # masking out "no data" region
maskedComposite = waterdata.updateMask(mask).subtract(1) # Shifting the labels to make it binary

## Setting up Config - to train globally

In total there are:
- 32 feature stack deep learning experiments
- 37 Multiview with 2 input experiments
- 8 Multiview with 3 input experiments

Each experiment has a different configuration for example, different experiment name, bands for each input layer. Hence, a configuration is neccesary. There is also configuration for training locally in Thailand and is found in Preprocessing_and_export.ipynb notebook

In [ ]:
configs_fs_global = {}
train_size = 72*10
eval_size = 72*3

#### Feature stack experiment

configs_fs_global["S1A_el_sl_as"] = config.configuration("S1A_el_sl_as", ["VV", "VH", "angle", "elevation", "slope", "aspect"], train_size, eval_size, country = "global")
configs_fs_global["S1A_el"] = config.configuration("S1A_el", ["VV", "VH", "angle", "elevation"], train_size, eval_size, country = "global")
configs_fs_global["S1A_sl"] = config.configuration("S1A_sl", ["VV", "VH", "angle", "slope"], train_size, eval_size, country = "global")
configs_fs_global["S1A_as"] = config.configuration("S1A_as", ["VV", "VH", "angle", "aspect"], train_size, eval_size, country = "global")
configs_fs_global["S1A_sl_as"] = config.configuration("S1A_sl_as", ["VV", "VH", "angle", "slope", "aspect"], train_size, eval_size, country = "global")
configs_fs_global["S1A_el_sl"] = config.configuration("S1A_el_sl", ["VV", "VH", "angle", "elevation", "slope"], train_size, eval_size, country = "global")
configs_fs_global["S1A_el_as"] = config.configuration("S1A_el_as", ["VV", "VH", "angle", "elevation", "aspect"], train_size, eval_size, country = "global")

configs_fs_global["S1_el_sl_as"] = config.configuration("S1_el_sl_as", ["VV", "VH", "elevation", "slope", "aspect"], train_size, eval_size, country = "global")
configs_fs_global["S1_el"] = config.configuration("S1_el", ["VV", "VH", "elevation"], train_size, eval_size, country = "global")
configs_fs_global["S1_sl"] = config.configuration("S1_sl", ["VV", "VH", "slope"], train_size, eval_size, country = "global")
configs_fs_global["S1_as"] = config.configuration("S1_as", ["VV", "VH", "aspect"], train_size, eval_size, country = "global")
configs_fs_global["S1_sl_as"] = config.configuration("S1_sl_as", ["VV", "VH", "slope", "aspect"], train_size, eval_size, country = "global")
configs_fs_global["S1_el_sl"] = config.configuration("S1_el_sl", ["VV", "VH", "elevation", "slope"], train_size, eval_size, country = "global")
configs_fs_global["S1_el_as"] = config.configuration("S1_el_as", ["VV", "VH", "elevation", "aspect"], train_size, eval_size, country = "global")

configs_fs_global["L8SR_el_sl_as"] = config.configuration("L8SR_el_sl_as", ["B2", "B3", "B4", "B5", "B6", "B7", "elevation", "slope", "aspect"], train_size, eval_size, country = "global")
configs_fs_global["L8SR_el"] = config.configuration("L8SR_el", ["B2", "B3", "B4", "B5", "B6", "B7", "elevation"], train_size, eval_size, country = "global")
configs_fs_global["L8SR_sl"] = config.configuration("L8SR_sl", ["B2", "B3", "B4", "B5", "B6", "B7", "slope"], train_size, eval_size, country = "global")
configs_fs_global["L8SR_as"] = config.configuration("L8SR_as", ["B2", "B3", "B4", "B5", "B6", "B7", "aspect"], train_size, eval_size, country = "global")
configs_fs_global["L8SR_sl_as"] = config.configuration("L8SR_sl_as", ["B2", "B3", "B4", "B5", "B6", "B7", "slope", "aspect"], train_size, eval_size, country = "global")
configs_fs_global["L8SR_el_sl"] = config.configuration("L8SR_el_sl", ["B2", "B3", "B4", "B5", "B6", "B7", "elevation", "slope"], train_size, eval_size, country = "global")
configs_fs_global["L8SR_el_as"] = config.configuration("L8SR_el_as", ["B2", "B3", "B4", "B5", "B6", "B7", "elevation", "aspect"], train_size, eval_size, country = "global")

configs_fs_global["L8SR_S1_el"] = config.configuration("L8SR_S1_el", ["B2", "B3", "B4", "B5", "B6", "B7", "VV", "VH", "elevation"], train_size, eval_size, country = "global")
configs_fs_global["L8SR_S1_sl"] = config.configuration("L8SR_S1_sl", ["B2", "B3", "B4", "B5", "B6", "B7", "VV", "VH", "slope"], train_size, eval_size, country = "global")
configs_fs_global["L8SR_S1_sl_el_as"] = config.configuration("L8SR_S1_sl_el_as", ["B2", "B3", "B4", "B5", "B6", "B7", "VV", "VH", "slope","elevation", "aspect"], train_size, eval_size, country = "global")

configs_fs_global["L8SR_S1A_el"] = config.configuration("L8SR_S1A_el", ["B2", "B3", "B4", "B5", "B6", "B7", "VV", "VH", "angle", "elevation"], train_size, eval_size, country = "global")
configs_fs_global["L8SR_S1A_sl"] = config.configuration("L8SR_S1A_sl", ["B2", "B3", "B4", "B5", "B6", "B7", "VV", "VH", "angle", "slope"], train_size, eval_size, country = "global")
configs_fs_global["L8SR_S1A_sl_el_as"] = config.configuration("L8SR_S1A_sl_el_as", ["B2", "B3", "B4", "B5", "B6", "B7", "VV", "VH", "angle", "slope","elevation", "aspect"], train_size, eval_size, country = "global")

configs_fs_global["L8SR"] = config.configuration("L8SR", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, country = "global")
configs_fs_global["S1"] = config.configuration("S1", ["VV", "VH"], train_size, eval_size, country = "global")
configs_fs_global["S1A"] = config.configuration("S1A", ["VV", "VH", "angle"], train_size, eval_size, country = "global")
configs_fs_global["L8SR_S1"] = config.configuration("L8SR_S1", ["B2", "B3", "B4", "B5", "B6", "B7", "VV", "VH"], train_size, eval_size, country = "global")
configs_fs_global["L8SR_S1A"] = config.configuration("L8SR_S1A", ["B2", "B3", "B4", "B5", "B6", "B7", "VV", "VH", "angle"], train_size, eval_size, country = "global")

###### Multiexperiment

configs_multi_global = {}

configs_multi_global["S1A_el_sl_as"] = config.configuration("S1A_el_sl_as", ["VV", "VH", "angle"], train_size, eval_size, ["elevation", "slope", "aspect"], type_=2, country = "global")
configs_multi_global["S1A_el"] = config.configuration("S1A_el", ["VV", "VH", "angle"], train_size, eval_size, ["elevation"], type_=2, country = "global")
configs_multi_global["S1A_sl"] = config.configuration("S1A_sl", ["VV", "VH", "angle"], train_size, eval_size, ["slope"], type_=2, country = "global")
configs_multi_global["S1A_as"] = config.configuration("S1A_as", ["VV", "VH", "angle"], train_size, eval_size, ["aspect"], type_=2, country = "global")
configs_multi_global["S1A_sl_as"] = config.configuration("S1A_sl_as", ["VV", "VH", "angle"], train_size, eval_size, ["slope", "aspect"], type_=2, country = "global")
configs_multi_global["S1A_el_sl"] = config.configuration("S1A_el_sl", ["VV", "VH", "angle"], train_size, eval_size, ["elevation", "slope"], type_=2, country = "global")
configs_multi_global["S1A_el_as"] = config.configuration("S1A_el_as", ["VV", "VH", "angle"], train_size, eval_size, ["elevation", "aspect"], type_=2, country = "global")

configs_multi_global["S1_el_sl_as"] = config.configuration("S1_el_sl_as", ["VV", "VH"], train_size, eval_size, ["elevation", "slope", "aspect"], type_=2, country = "global")
configs_multi_global["S1_el"] = config.configuration("S1_el", ["VV", "VH"], train_size, eval_size, ["elevation"], type_=2, country = "global")
configs_multi_global["S1_sl"] = config.configuration("S1_sl", ["VV", "VH"], train_size, eval_size, ["slope"], type_=2, country = "global")
configs_multi_global["S1_as"] = config.configuration("S1_as", ["VV", "VH"], train_size, eval_size, ["aspect"], type_=2, country = "global")
configs_multi_global["S1_sl_as"] = config.configuration("S1_sl_as", ["VV", "VH"], train_size, eval_size, ["slope", "aspect"], type_=2, country = "global")
configs_multi_global["S1_el_sl"] = config.configuration("S1_el_sl", ["VV", "VH"], train_size, eval_size, ["elevation", "slope"], type_=2, country = "global")
configs_multi_global["S1_el_as"] = config.configuration("S1_el_as", ["VV", "VH"], train_size, eval_size, ["elevation", "aspect"], type_=2, country = "global")

configs_multi_global["L8SR_el_sl_as"] = config.configuration("L8SR_el_sl_as", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["elevation", "slope", "aspect"], type_=2, country = "global")
configs_multi_global["L8SR_el"] = config.configuration("L8SR_el", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["elevation"], type_=2, country = "global")
configs_multi_global["L8SR_sl"] = config.configuration("L8SR_sl", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["slope"], type_=2, country = "global")
configs_multi_global["L8SR_as"] = config.configuration("L8SR_as", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["aspect"], type_=2, country = "global")
configs_multi_global["L8SR_sl_as"] = config.configuration("L8SR_sl_as", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["slope", "aspect"], type_=2, country = "global")
configs_multi_global["L8SR_el_sl"] = config.configuration("L8SR_el_sl", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["elevation", "slope"], type_=2, country = "global")
configs_multi_global["L8SR_el_as"] = config.configuration("L8SR_el_as", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["elevation", "aspect"], type_=2, country = "global")


configs_multi_global["L8SR_S1_as"] = config.configuration("L8SR_S1_as", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "aspect"], type_=2, country = "global")
configs_multi_global["L8SR_S1_el"] = config.configuration("L8SR_S1_el", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "elevation"], type_=2, country = "global")
configs_multi_global["L8SR_S1_sl"] = config.configuration("L8SR_S1_sl", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "slope"], type_=2, country = "global")

configs_multi_global["L8SR_S1_sl_as"] = config.configuration("L8SR_S1_sl_as", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "slope", "aspect"], type_=2, country = "global")
configs_multi_global["L8SR_S1_el_sl"] = config.configuration("L8SR_S1_el_sl", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "elevation", "slope"], type_=2, country = "global")
configs_multi_global["L8SR_S1_el_as"] = config.configuration("L8SR_S1_el_as", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "elevation", "aspect"], type_=2, country = "global")
configs_multi_global["L8SR_S1_sl_el_as"] = config.configuration("L8SR_S1_sl_el_as", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "slope","elevation", "aspect"], type_=2, country = "global")

configs_multi_global["L8SR_S1A_as"] = config.configuration("L8SR_S1A_as", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "angle", "aspect"], type_=2, country = "global")
configs_multi_global["L8SR_S1A_el"] = config.configuration("L8SR_S1A_el", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "angle", "elevation"], type_=2, country = "global")
configs_multi_global["L8SR_S1A_sl"] = config.configuration("L8SR_S1A_sl", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "angle", "slope"], type_=2, country = "global")

configs_multi_global["L8SR_S1A_sl_as"] = config.configuration("L8SR_S1A_sl_as", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "angle", "slope", "aspect"], type_=2, country = "global")
configs_multi_global["L8SR_S1A_el_sl"] = config.configuration("L8SR_S1A_el_sl", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "angle", "elevation", "slope"], type_=2, country = "global")
configs_multi_global["L8SR_S1A_el_as"] = config.configuration("L8SR_S1A_el_as", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "angle", "elevation", "aspect"], type_=2, country = "global")
configs_multi_global["L8SR_S1A_sl_el_as"] = config.configuration("L8SR_S1A_sl_el_as", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "angle", "slope","elevation", "aspect"], type_=2, country = "global")

configs_multi_global["L8SR_S1"] = config.configuration("L8SR_S1", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH"], type_=2, country = "global")
configs_multi_global["L8SR_S1A"] = config.configuration("L8SR_S1A", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "angle"], type_=2, country = "global")

configs_multi_3_global = {}

configs_multi_3_global["L8SR_S1_as3"] = config.configuration("L8SR_S1_as3", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH"], ["aspect"], type_=3, country = "global")
configs_multi_3_global["L8SR_S1_el3"] = config.configuration("L8SR_S1_el3", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH"], ["elevation"], type_=3, country = "global")
configs_multi_3_global["L8SR_S1_sl3"] = config.configuration("L8SR_S1_sl3", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH"], ["slope"], type_=3, country = "global")
configs_multi_3_global["L8SR_S1_sl_el_as3"] = config.configuration("L8SR_S1_sl_el_as3", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH"], ["slope","elevation", "aspect"], type_=3, country = "global")

configs_multi_3_global["L8SR_S1A_as3"] = config.configuration("L8SR_S1A_as3", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "angle"], ["aspect"], type_=3, country = "global")
configs_multi_3_global["L8SR_S1A_el3"] = config.configuration("L8SR_S1A_el3", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "angle"], ["elevation"], type_=3, country = "global")
configs_multi_3_global["L8SR_S1A_sl3"] = config.configuration("L8SR_S1A_sl3", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "angle"], ["slope"], type_=3, country = "global")
configs_multi_3_global["L8SR_S1A_sl_el_as3"] = config.configuration("L8SR_S1A_sl_el_as3", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "angle"], ["slope","elevation", "aspect"], type_=3)

In [ ]:
print(configs_multi_global["L8SR_S1A"].FOLDER)
print(configs_multi_global["L8SR_S1A"].BANDS1)
print(configs_multi_global["L8SR_S1A"].BANDS2)
print(configs_multi_global["L8SR_S1A"].BANDS)
print(configs_multi_global["L8SR_S1A"].TRAINING_BASE)
print(configs_multi_global["L8SR_S1A"].EVAL_BASE)
print(configs_multi_global["L8SR_S1A"].TEST_BASE_1)
print(configs_multi_global["L8SR_S1A"].TEST_BASE_2)
print(configs_multi_global["L8SR_S1A"].country)
print(configs_multi_global["L8SR_S1A"].type_)

m2_global_Cnn_L8SR_S1A
['B2', 'B3', 'B4', 'B5', 'B6', 'B7']
['VV', 'VH', 'angle']
['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'VV', 'VH', 'angle']
training_patches
eval_patches
test_patches_1
test_patches_2
global
m2


We also need to add the images to the configuration. This will be neccesary when we want to export images to google earth assets later on.

In [ ]:
# Feature Stack

configs_fs_global["S1A_el_sl_as"].image = ee.Image.cat([S1A, elevation, slope, aspect]).float()
configs_fs_global["S1A_el"].image = ee.Image.cat([S1A, elevation]).float()
configs_fs_global["S1A_sl"].image = ee.Image.cat([S1A, slope]).float()
configs_fs_global["S1A_as"].image = ee.Image.cat([S1A, aspect]).float()
configs_fs_global["S1A_sl_as"].image = ee.Image.cat([S1A, slope, aspect]).float()
configs_fs_global["S1A_el_sl"].image = ee.Image.cat([S1A, elevation, slope]).float()
configs_fs_global["S1A_el_as"].image = ee.Image.cat([S1A, elevation, aspect]).float()

configs_fs_global["S1_el_sl_as"].image = ee.Image.cat([S1, elevation, slope, aspect]).float()
configs_fs_global["S1_el"].image = ee.Image.cat([S1, elevation]).float()
configs_fs_global["S1_sl"].image = ee.Image.cat([S1, slope]).float()
configs_fs_global["S1_as"].image = ee.Image.cat([S1, aspect]).float()
configs_fs_global["S1_sl_as"].image = ee.Image.cat([S1, slope, aspect]).float()
configs_fs_global["S1_el_sl"].image = ee.Image.cat([S1, elevation, slope]).float()
configs_fs_global["S1_el_as"].image = ee.Image.cat([S1, elevation, aspect]).float()

configs_fs_global["L8SR_el_sl_as"].image = ee.Image.cat([L8SR, elevation, slope, aspect]).float()
configs_fs_global["L8SR_el"].image = ee.Image.cat([L8SR, elevation]).float()
configs_fs_global["L8SR_sl"].image = ee.Image.cat([L8SR, slope]).float()
configs_fs_global["L8SR_as"].image = ee.Image.cat([L8SR, aspect]).float()
configs_fs_global["L8SR_sl_as"].image = ee.Image.cat([L8SR, slope, aspect]).float()
configs_fs_global["L8SR_el_sl"].image = ee.Image.cat([L8SR, elevation, slope]).float()
configs_fs_global["L8SR_el_as"].image = ee.Image.cat([L8SR, elevation, aspect]).float()

configs_fs_global["L8SR_S1_el"].image = ee.Image.cat([L8SR, S1, elevation]).float()
configs_fs_global["L8SR_S1_sl"].image = ee.Image.cat([L8SR, S1, slope]).float()
configs_fs_global["L8SR_S1_sl_el_as"].image = ee.Image.cat([L8SR, S1, slope, elevation, aspect]).float()

configs_fs_global["L8SR_S1A_el"].image = ee.Image.cat([L8SR, S1A, elevation]).float()
configs_fs_global["L8SR_S1A_sl"].image = ee.Image.cat([L8SR, S1A, slope]).float()
configs_fs_global["L8SR_S1A_sl_el_as"].image = ee.Image.cat([L8SR, S1A, slope, elevation, aspect]).float()

configs_fs_global["L8SR"].image = L8SR.float()
configs_fs_global["S1"].image = S1.float()
configs_fs_global["S1A"].image = S1A.float()
configs_fs_global["L8SR_S1"].image = ee.Image.cat([L8SR, S1]).float()
configs_fs_global["L8SR_S1A"].image = ee.Image.cat([L8SR, S1A]).float()


# Multiview

configs_multi_global["S1A_el_sl_as"].image = ee.Image.cat([S1A, elevation, slope, aspect]).float() 
configs_multi_global["S1A_el"].image = ee.Image.cat([S1A, elevation]).float() 
configs_multi_global["S1A_sl"].image = ee.Image.cat([S1A, slope]).float() 
configs_multi_global["S1A_as"].image = ee.Image.cat([S1A, aspect]).float() 
configs_multi_global["S1A_sl_as"].image = ee.Image.cat([S1A, slope, aspect]).float() 
configs_multi_global["S1A_el_sl"].image = ee.Image.cat([S1A, elevation, slope]).float() 
configs_multi_global["S1A_el_as"].image = ee.Image.cat([S1A, elevation, aspect]).float() 

configs_multi_global["S1_el_sl_as"].image = ee.Image.cat([S1, elevation, slope, aspect]).float()  
configs_multi_global["S1_el"].image = ee.Image.cat([S1, elevation]).float()
configs_multi_global["S1_sl"].image = ee.Image.cat([S1, slope]).float() 
configs_multi_global["S1_as"].image = ee.Image.cat([S1, aspect]).float() 
configs_multi_global["S1_sl_as"].image = ee.Image.cat([S1, slope, aspect]).float()
configs_multi_global["S1_el_sl"].image = ee.Image.cat([S1, elevation, slope]).float() 
configs_multi_global["S1_el_as"].image = ee.Image.cat([S1, elevation, aspect]).float() 

configs_multi_global["L8SR_el_sl_as"].image = ee.Image.cat([L8SR, elevation, slope, aspect]).float() 
configs_multi_global["L8SR_el"].image = ee.Image.cat([L8SR, elevation]).float() 
configs_multi_global["L8SR_sl"].image = ee.Image.cat([L8SR, slope]).float() 
configs_multi_global["L8SR_as"].image = ee.Image.cat([L8SR, aspect]).float() 
configs_multi_global["L8SR_sl_as"].image = ee.Image.cat([L8SR, slope, aspect]).float() 
configs_multi_global["L8SR_el_sl"].image = ee.Image.cat([ L8SR, elevation, slope]).float() 
configs_multi_global["L8SR_el_as"].image = ee.Image.cat([L8SR, elevation, aspect]).float() 


configs_multi_global["L8SR_S1_as"].image = ee.Image.cat([L8SR, S1, aspect]).float()
configs_multi_global["L8SR_S1_el"].image = ee.Image.cat([L8SR, S1, elevation]).float() 
configs_multi_global["L8SR_S1_sl"].image = ee.Image.cat([L8SR, S1, slope]).float() 

configs_multi_global["L8SR_S1_sl_as"].image = ee.Image.cat([L8SR, S1, slope, aspect]).float() 
configs_multi_global["L8SR_S1_el_sl"].image = ee.Image.cat([L8SR, S1, elevation, slope]).float() 
configs_multi_global["L8SR_S1_el_as"].image = ee.Image.cat([L8SR, slope, elevation, aspect]).float() 
configs_multi_global["L8SR_S1_sl_el_as"].image = ee.Image.cat([L8SR, S1, slope, elevation, aspect]).float() 

configs_multi_global["L8SR_S1A_as"].image = ee.Image.cat([L8SR, S1A, aspect]).float()
configs_multi_global["L8SR_S1A_el"].image = ee.Image.cat([L8SR, S1A, elevation]).float() 
configs_multi_global["L8SR_S1A_sl"].image = ee.Image.cat([L8SR, S1A, slope]).float() 

configs_multi_global["L8SR_S1A_sl_as"].image = ee.Image.cat([L8SR, S1A, slope, aspect]).float() 
configs_multi_global["L8SR_S1A_el_sl"].image = ee.Image.cat([L8SR, S1A, elevation, slope]).float() 
configs_multi_global["L8SR_S1A_el_as"].image = ee.Image.cat([L8SR, S1A, elevation, aspect]).float() 
configs_multi_global["L8SR_S1A_sl_el_as"].image = ee.Image.cat([L8SR, S1A, slope, elevation, aspect]).float() 

configs_multi_global["L8SR_S1"].image = ee.Image.cat([L8SR, S1]).float() 
configs_multi_global["L8SR_S1A"].image = ee.Image.cat([L8SR, S1A]).float() 

# Multiview-3


configs_multi_3_global["L8SR_S1_as3"].image = ee.Image.cat([L8SR, S1, aspect])
configs_multi_3_global["L8SR_S1_el3"].image = ee.Image.cat([L8SR, S1, elevation])
configs_multi_3_global["L8SR_S1_sl3"].image = ee.Image.cat([L8SR, S1, slope])
configs_multi_3_global["L8SR_S1_sl_el_as3"].image = ee.Image.cat([L8SR, S1, slope, elevation, aspect])

configs_multi_3_global["L8SR_S1A_as3"].image = ee.Image.cat([L8SR, S1A, aspect])
configs_multi_3_global["L8SR_S1A_el3"].image = ee.Image.cat([L8SR, S1A, elevation])
configs_multi_3_global["L8SR_S1A_sl3"].image = ee.Image.cat([L8SR, S1A, slope])
configs_multi_3_global["L8SR_S1A_sl_el_as3"].image = ee.Image.cat([L8SR, S1A, slope, elevation, aspect])


We stack the 2D images (Landsat composite and JRC water data) to create a single image from which samples can be taken. We convert the image into an array image in which each pixel stores 256x256 patches of pixels for each band.  To export training patches, convert a multi-band image to an array image using neighborhoodToArray(), then sample the image at points.

In [ ]:
for key in list(configs_fs_global):
  settings = configs_fs_global[key]
  featureStack = ee.Image.cat([
  settings.image.select(settings.BANDS),
  maskedComposite.select(settings.RESPONSE)
  ]).float()
  list_ = ee.List.repeat(1, settings.KERNEL_SIZE)
  lists = ee.List.repeat(list_, settings.KERNEL_SIZE)
  kernel = ee.Kernel.fixed(settings.KERNEL_SIZE, settings.KERNEL_SIZE, lists)
  arrays = featureStack.neighborhoodToArray(kernel)
  configs_fs_global[key].sam_arr = arrays
  print(key, settings.sam_arr.getInfo())
  

S1A_el_sl_as {'type': 'Image', 'bands': [{'id': 'VV', 'data_type': {'type': 'PixelType', 'precision': 'float', 'dimensions': 2}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'VH', 'data_type': {'type': 'PixelType', 'precision': 'float', 'dimensions': 2}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'angle', 'data_type': {'type': 'PixelType', 'precision': 'float', 'dimensions': 2}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'elevation', 'data_type': {'type': 'PixelType', 'precision': 'float', 'dimensions': 2}, 'dimensions': [1288801, 421201], 'crs': 'EPSG:4326', 'crs_transform': [0.0002777777777777778, 0, -179.0001388888889, 0, -0.0002777777777777778, 61.00013888888889]}, {'id': 'slope', 'data_type': {'type': 'PixelType', 'precision': 'float', 'dimensions': 2}, 'crs': 'EPSG:4326', 'crs_transform': [0.0002777777777777778, 0, -179.0001388888889, 0, -0.0002777777777777778, 61.00013888888889]}, {'id': 'aspect', 'data_type': {'typ

# Sampling

## Sampling globally

We create some geometries to sample the stack in strategic locations to take the 256x256 samples.  Display the sampling polygons on a map, red for training polygons, blue for evaluation and green for testing polygons

In [ ]:
India = ee.Geometry.BBox(84.28766453687791, 24.192675258975537, 87.49567234937791, 26.180892485228437)
Pakistan = ee.Geometry.BBox(70.3009325419945, 31.3604931755229, 71.5533739482445, 32.33570151710531)
Cambodia = ee.Geometry.BBox(105.35157579365571, 12.08228250180923, 106.32935899678071, 13.025952639601176)
Brazil = ee.Geometry.BBox(-60.060403802711626, -19.002700742809683, -53.204935052711626, -13.90505097174063)
Bangladesh = ee.Geometry.BBox(89.50323560069529, 23.090980419149876, 91.00286938975779, 24.077692617901942)
Mexico = ee.Geometry.BBox(-104.87430997262705, 19.986643103978352, -98.01884122262705, 26.32579195587412)
Egypt = ee.Geometry.BBox(30.120884854535284, 17.6368163032104, 34.169346768597784, 19.754417837576597)
Thailand = ee.Geometry.BBox(100.47883808661508, 5.9400179647667635, 101.59479267596963, 6.8385138873107945)
Ghana = ee.Geometry.BBox(-1.0995263958226564, 6.221477061378518, 0.44954586980234357, 7.639227255737275)
Tibet = ee.Geometry.BBox(88.13943211306636, 28.727971943391157, 91.17165867556636, 31.04238968021801)


trainingPolys_global = ee.FeatureCollection(India).merge(Pakistan).merge(Cambodia).merge(Brazil) \
                          .merge(Bangladesh).merge(Mexico).merge(Egypt).merge(Thailand).merge(Ghana) \
                          .merge(Tibet)

India_eval = ee.Geometry.BBox(75.68466076783874, 21.263314014804795, 78.04672131471374, 22.770654917923526)
Tibet_eval = ee.Geometry.BBox(93.92188430197075, 26.847874650118836, 95.93238234884575, 28.356873608833094)
brazil_eval = ee.Geometry.BBox(-53.749924908880686, -22.833543783979856, -51.047288190130686, -20.999024800255494)

evalPolys_global = ee.FeatureCollection(Tibet_eval).merge(India_eval).merge(brazil_eval)

thai_test = ee.Geometry.BBox(100.30632852425321, 17.709225431372587, 100.74128946175321, 18.20417872756825)
tibet_test = ee.Geometry.BBox(83.7866460908476, 31.02991423438545, 84.4782964814726, 31.623526673040716)
ghana_test = ee.Geometry.BBox(-1.9983132238272127, 5.925449378444892, -1.8517086095694002, 6.081004042776062)
brazil_test = ee.Geometry.BBox(-63.02141262744682, -28.962951804200575, -55.59465481494682, -21.415208673846603)
mexico_test = ee.Geometry.BBox(-93.47672355917602, 15.8775670491606535, -90.70816887167602, 18.27183442641013)
pakistan_test = ee.Geometry.BBox(68.55305064706133, 27.98976293885938, 70.93708385018633, 29.42545415991563)
egypt_test = ee.Geometry.BBox(24.8505863665, 20.337905952546933, 33.1123051165, 26.856885967831754)
cambodia_test = ee.Geometry.BBox(102.74910463260488, 11.950119301574833, 104.88045228885488, 13.600069066335008)
India_test = ee.Geometry.BBox(80.42186850663656, 22.601971814608234, 83.05858725663656, 24.635163539106795)
Bangladesh_test = ee.Geometry.BBox(89.25478625914253, 24.205326578074743, 90.15566516539253, 24.765228419516816)

testPolys_global = ee.FeatureCollection(thai_test).merge(tibet_test).merge(ghana_test).merge(brazil_test) \
                          .merge(mexico_test).merge(pakistan_test).merge(egypt_test).merge(cambodia_test).merge(India_test) \
                          .merge(Bangladesh_test)

polyImage = ee.Image(0).byte().paint(trainingPolys_global, 1).paint(evalPolys_global, 2).paint(testPolys_global, 3)
polyImage = polyImage.updateMask(polyImage)

mapid = polyImage.getMapId({'min': 1, 'max': 3, 'palette': ['red', 'blue', "green"]})
map = folium.Map(location=[16.426385350573945, 102.16833965985882], zoom_start=5)
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='training polygons',
  ).add_to(map)
map.add_child(folium.LayerControl())
map

sampling.py file will help with sampling by merging smaller samples within each geometry into a single export to avoid the 'computed value too large' error. The function will sample the array image at points, to get all the pixels in a 256x256 neighborhood at each point. Each rectangle will get exported as a *.tfrecord.gz

Below we export the data for training, evaluation and testing. The training and evaluation data is done globally. For training data export in Thailand look at `Preprocessing_and_export.ipynb`. The total number of training data is 720 which is the same as in `Preprocessing_and_export.ipynb` for a fair experiment

The testing data sampled in 10 different location and is the same location as in `Preprocessing_and_export_global.ipynb`

In [ ]:
n = 6 # Number of shards in each polygon.
N = 72 # Total sample size in each polygon.
settings = configs_fs_global["L8SR_S1A_sl_el_as"] # This contains all the features
sampling.Training_task(trainingPolys_global, n, N, settings.sam_arr, settings, "train_in_global")
sampling.Eval_task(evalPolys_global, n, N, settings.sam_arr, settings, "train_in_global")
sampling.Testing_task(testPolys_global, n, N, settings.sam_arr, settings, "train_in_global", settings.TEST_BASE)

The following code is to track the progress of exporting or cancelling tasks, if wrong data is exported

In [ ]:
# Print all tasks.
# pprint(ee.batch.Task.list())
for i in range(2):
  pprint(ee.batch.Task.list()[i])

<Task 5H4ORMD6CID53HARL5CVDBJJ INGEST_IMAGE: Ingest image: "projects/earthengine-legacy/assets/users/mewchayutaphong/Ghana_with_Multi_2L8SR_S1A_sl_epochs_10_" (COMPLETED)>
<Task RB5FRZAOSHONNJJFTDA23M4D EXPORT_IMAGE: Ghana_with_Multi_2L8SR_S1A_sl (COMPLETED)>


In [ ]:
# !earthengine task cancel all

Canceling task "L5ZJN2IZ6YKWXLD5Q6HUEHOJ"
Canceling task "CKQZODRNXSSOLMPQAMGLR7TW"
Canceling task "KL2B7QTZSZHUDEE73HROSIGS"
Canceling task "43HMONZIVDVGYVQX32LIUSV2"
Canceling task "ISZBI4CHWZGG22O5JNIEHJCZ"
Canceling task "7IOL6STFSKJXBKPAXFTFDEZY"
Canceling task "4NOWTSSAXGPOTN7AA4N2WIDS"
Canceling task "TSD4VGQ5TQ2KL7WBRA2BVWAA"
Canceling task "3VNLRTAVH47YAE3M652SD2SN"
Canceling task "WZRXSTM76TEIVYONKINE4RSW"
Canceling task "6JRDYC4DTPPOZGD4UXY7MDWT"
Canceling task "PWDQER5WCU3YOBVCFVI52TRK"


## Prediction whole countries

In [ ]:
Thailand = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')\
              .filter(ee.Filter.eq('country_na', 'Thailand'));

Bangladesh = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')\
              .filter(ee.Filter.eq('country_na', 'Bangladesh'));
              
Mexico = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')\
              .filter(ee.Filter.eq('country_na', 'Mexico'));

Australia = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')\
              .filter(ee.Filter.eq('country_na', 'Australia'));


India = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')\
              .filter(ee.Filter.eq('country_na', 'India'));
              

Vietnam = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')\
              .filter(ee.Filter.eq('country_na', 'Vietnam'));

Egypt = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')\
              .filter(ee.Filter.eq('country_na', 'Egypt'));

Pakistan = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')\
              .filter(ee.Filter.eq('country_na', 'Pakistan'));

Indonesia = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')\
              .filter(ee.Filter.eq('country_na', 'Indonesia'));

Iran = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')\
              .filter(ee.Filter.eq('country_na', 'Iran'));

Qatar = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')\
              .filter(ee.Filter.eq('country_na', 'Qatar'));

Israel = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')\
              .filter(ee.Filter.eq('country_na', 'Israel'));

Lebanon = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')\
              .filter(ee.Filter.eq('country_na', 'Lebanon'));

Nepal = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')\
              .filter(ee.Filter.eq('country_na', 'Nepal'));

China = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')\
              .filter(ee.Filter.eq('country_na', 'China'));

USA = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')\
              .filter(ee.Filter.eq('country_na', 'United States'));

sAfrica = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')\
              .filter(ee.Filter.eq('country_na', 'South Africa'));
              
Brazil = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')\
              .filter(ee.Filter.eq('country_na', 'Brazil'));

Nigeria = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')\
              .filter(ee.Filter.eq('country_na', 'Nigeria'));

test_global = Thailand.merge(Bangladesh).merge(Mexico) \
                  .merge(Australia).merge(India).merge(Vietnam) \
                  .merge(Egypt).merge(Pakistan).merge(Indonesia) \
                  .merge(Iran).merge(Qatar).merge(Israel).merge(Lebanon) \
                  .merge(Nepal).merge(China).merge(USA) \
                  .merge(Brazil)

In [ ]:
n = 6 # Number of shards in each polygon.
N = 72 # Total sample size in each polygon.
settings = configs_fs_global["L8SR_S1A_sl_el_as"] # This contains all the features
sampling.Testing_task(test_global, n, N, settings.sam_arr, settings, "final_global_test", settings.TEST_BASE)

In [ ]:
# Print all tasks.
pprint(ee.batch.Task.list())
# for i in range(40):
#   pprint(ee.batch.Task.list()[i])

[<Task MB3SIOINZZKOBGTH5O2VVNQT EXPORT_FEATURES: test_patches_g23 (RUNNING)>,
 <Task CK3WZBGCGLEXZFDK47Z4J52I EXPORT_FEATURES: test_patches_g22 (COMPLETED)>,
 <Task Q3J6LQ34NGUSZEZTEB2POUO5 EXPORT_FEATURES: test_patches_g21 (COMPLETED)>,
 <Task 4KNFE5KGNCM63NIMYDI7B5BU EXPORT_FEATURES: test_patches_g20 (COMPLETED)>,
 <Task T2GYDBCO3ZYYR2ORP7XPXEEJ EXPORT_FEATURES: test_patches_g19 (COMPLETED)>,
 <Task H4FP2SXWLPE7EMBHFRR7GCBG EXPORT_FEATURES: test_patches_g18 (COMPLETED)>,
 <Task PP3FMNWXJJUQZREE6RQLQ6TH EXPORT_FEATURES: test_patches_g17 (COMPLETED)>,
 <Task ZX777QPPLK4TKT247VW24IYK EXPORT_FEATURES: test_patches_g16 (COMPLETED)>,
 <Task 5F6NDXMZ2ETGVVGLNJTPUSHT EXPORT_FEATURES: test_patches_g15 (COMPLETED)>,
 <Task MBD7M3OD3ELLPYP6LSYDJTAD EXPORT_FEATURES: test_patches_g14 (COMPLETED)>,
 <Task CJNXIVBQOGSZEJAOEU5YG6LM EXPORT_FEATURES: test_patches_g13 (COMPLETED)>,
 <Task 7BOF27A52ZAKIV5PTIPJHL74 EXPORT_FEATURES: test_patches_g12 (COMPLETED)>,
 <Task TCBYAFXZBAUKTCHBCU6IO3OP EXPORT_FEA

In [ ]:
!earthengine task cancel all

Canceling task "LHW2L3MOWNYB5WALIM4OWVT4"
Canceling task "MTBPUTESCAKWQQQUPFER6WEQ"
Canceling task "AFBLHQJW7U6O7KDHACJJAZPM"
Canceling task "QOWALS7SE3MPJ4B7HRUQHYO6"
Canceling task "GW6EJJZVV2GVA5PCIUALUKIP"
Canceling task "EQ3MXDHMCSFGADJCGOWE3635"
Canceling task "DFTPNOH6LDJNW32GVCDKGXW2"
Canceling task "FC5FFFICG7FWPOS3JJYSLOVU"
Canceling task "IKZ5W4TLGYILSAYEE6V7DGCG"
Canceling task "VIVFGCQLTUS3OMG7H6A7BGXX"
Canceling task "R24PR2Z3H2CV3DH2H4I6RFOJ"
Canceling task "DFYKRABFQFCDVAT34FW7J4BS"
Canceling task "5A46AVEF3ZAORCV2AWYFCAZN"
Canceling task "AG2I56WHVZUEH27UEYZRMJCK"
Canceling task "5AT2KNMR7HUP3AVWTXING72L"
Canceling task "SD6EBQXY2TSK3EBNNX7UZXR3"
Canceling task "WYBK3DLMD4F5JUHKJSDUV5JC"
Canceling task "IUQNXP2GWDSKDTSGE6EMV3HV"
Canceling task "CKAJYE6QLO3JSFPQLI3SJOJI"
Canceling task "KY3PZZDASIZ2XUJKR5DSN67O"
Canceling task "IUUVQ3CDV3LO4UFHPS64R62J"
Canceling task "GDURNN73ALX5TL6KKREJTGTG"
Canceling task "2ZQC5EOOBZ4FZCX4Y7RFSEBQ"
Canceling task "NGNRSAVGC7ZSLCSESF